# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa384f94610>
├── 'a' --> tensor([[ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106]])
└── 'x' --> <FastTreeValue 0x7fa400059520>
    └── 'c' --> tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                        [-0.2028,  2.0119, -0.0718,  0.3055],
                        [ 0.7258, -0.4089,  0.0210,  0.4837]])

In [4]:
t.a

tensor([[ 0.2516, -0.2525,  1.0908],
        [-0.2790,  0.4182,  1.7106]])

In [5]:
%timeit t.a

77.5 ns ± 0.281 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa384f94610>
├── 'a' --> tensor([[ 0.9925,  0.5216, -0.4160],
│                   [ 0.2290,  1.6712, -0.4907]])
└── 'x' --> <FastTreeValue 0x7fa400059520>
    └── 'c' --> tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                        [-0.2028,  2.0119, -0.0718,  0.3055],
                        [ 0.7258, -0.4089,  0.0210,  0.4837]])

In [7]:
%timeit t.a = new_value

78.2 ns ± 0.394 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106]]),
    x: Batch(
           c: tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                      [-0.2028,  2.0119, -0.0718,  0.3055],
                      [ 0.7258, -0.4089,  0.0210,  0.4837]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2516, -0.2525,  1.0908],
        [-0.2790,  0.4182,  1.7106]])

In [11]:
%timeit b.a

69.4 ns ± 0.298 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6070,  0.0178, -0.0819],
               [-0.8246, -2.4702, -1.1162]]),
    x: Batch(
           c: tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                      [-0.2028,  2.0119, -0.0718,  0.3055],
                      [ 0.7258, -0.4089,  0.0210,  0.4837]]),
       ),
)

In [13]:
%timeit b.a = new_value

582 ns ± 4.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

984 ns ± 3.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 48.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

164 µs ± 605 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

164 µs ± 707 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa384dc6790>
├── 'a' --> tensor([[[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]],
│           
│                   [[ 0.2516, -0.2525,  1.0908],
│                    [-0.2790,  0.4182,  1.7106]]])
└── 'x' --> <FastTreeValue 0x7fa37e100f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.7 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa384d554f0>
├── 'a' --> tensor([[ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106],
│                   [ 0.2516, -0.2525,  1.0908],
│                   [-0.2790,  0.4182,  1.7106]])
└── 'x' --> <FastTreeValue 0x7fa40046e6d0>
    └── 'c' --> tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                        [-0.2028,  2.0119, -0.0718,  0.3055],
                 

In [23]:
%timeit t_cat(trees)

34.8 µs ± 77.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.2 µs ± 249 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]],
       
               [[ 0.2516, -0.2525,  1.0908],
                [-0.2790,  0.4182,  1.7106]]]),
    x: Batch(
           c: tensor([[[ 1.8520,  0.5151,  0.0689, -0.1498],
                       [-0.2028,  2.0119, -0.0718,  0.3055],
                       [ 0.7258, -0.4089,  0.0210,  0.4837]],
         

In [26]:
%timeit Batch.stack(batches)

90.8 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106],
               [ 0.2516, -0.2525,  1.0908],
               [-0.2790,  0.4182,  1.7106]]),
    x: Batch(
           c: tensor([[ 1.8520,  0.5151,  0.0689, -0.1498],
                      [-0.2028,  2.0119, -0.0718,  0.3055],
                      [ 0.7258, -0.4089,  0.0210,  0.4837],
                      [ 1.8520,  0.5151,  0.0689, -0.1498],
                      [-0.2028,  

In [28]:
%timeit Batch.cat(batches)

165 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

399 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
